# OOP - Part IV.b: Abstract Classes

In Lecture 18a, we have seen how to use inheritance in the context of *concrete* classes - i.e. classes whose instances can be created (such as `HomoSapien`, `LivingBeing` etc.). There is another context in which inheritance proves valuable: *abstract* classes. An abstract class is a class whose instances cannot be created. Typically, these classes leave the functionality of some methods unspecified. 

The only use of these classes is in the context of inheritance - we create a *derived* class that inherits from such an abstract class. If the derived class defines the functionality of all methods in the abstract class, we can create its instance. If functionality of some methods remains unspecified, the derived class is also abstract and we cannot create its instances. 

Abstract classes are useful when we wish to specify the syntax of some (or all) methods, but do not wish to implement the method or restrict to one specific implementation. We will see examples that arise naturally in the context of Mathematics. 

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* Abstract classes and abstract methods
* Inheriting from abstract classes
* Implementing abstract methods
* Decorators

<hr style="height: 2px">

*&#169; Pranav Singh, University of Bath 2021-2022. This problem sheet is copyright of Pranav Singh, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

# Abstract Classes and Abstract Methods

A Ring $(A, +, \cdot, 0, 1)$ is a set $A$ along with two operators: $+$ and $\cdot$. 

* The set $A$ has an element $0$ which is an additive identity 
* The set $A$ has an element $1$ which is a multiplicative identity.  
* For every $a \in A$, there is an element $-a \in A$ (called additive inverse) such that $a + (-a) = 0$.
* We can add two elements: $a+b$
* We can subtract an element from another: $a-b = a+(-b)$. 
* We can multiply two elements: $a \cdot b$. 
* In general there does not have to be a multiplicative inverse $a^{-1}$ and division $a/b$ is not necessarily defined. 
* We can always define non-negative integer powers of any element since $a^0 = 1$ and $a^{n+1} = a \cdot (a^n)$.

The following are some examples of rings:

* $(\mathbb{Q}, +, \cdot, 0, 1)$ the ring of rationals.
* $(\mathbb{C}, +, \cdot, 0, 1)$ the ring of complex numbers.
* $(\mathbb{R}^{n\times n}, +, \cdot, O_n, I_n)$ the ring of $n \times n$ real-valued matrices, where $A \cdot B$ is matrix product.
* $(\mathcal{P}(x), +, \cdot, 0, 1)$ the ring of polynomials in $x$.

Thus, many different classes could act as a *ring* and it does not make sense to implement `Ring` as a *concrete* class. Instead, `Ring` should be implemented as an *abstract* class and the above 4 examples should be concrete examples derived from it. `Ring` should specify *syntax* (not implementation) of methods and operations such as addition and multiplication, while the concrete classes should implement these operations.

In Python, we create an abstract class by using Abstract Base Classes (ABC). An Abstract Base Class should not be used directly for creating instances. Rather, the purpose of an ABC is to specify the functionality that other classes should inherit and implement. 

We create an Abstract Base Class called `Ring` by inheriting from the class `ABC` (imported from package `abc`). 

In [ ]:
from abc import ABC, abstractmethod

class Ring(ABC):
    '''Ring (A, +, ., zero, id)'''
    
    @abstractmethod
    def __mul__(a,b):
        ...
    
    @abstractmethod
    def __add__(a,b):
        ...
        
    @abstractmethod
    def __neg__(a,b):    
        ...
    
    # Even if we do not know how to add or how to negate,
    # we can already define subtraction in terms of these operators.
    def __sub__(a,b):
        return a+(-b)  # Note that a+(-b) is equivalent to a.__add__(b.__neg__())
    
    # The multiplicative identity "1" is written as "id" since 1 is reserved for the integer 
    @abstractmethod
    def id():
        ...
    
    # The additive identity "0" is written as "zero" since 0 is reserved for the integer 
    @abstractmethod
    def zero():
        ...
    
    @abstractmethod
    def __str__(self):
        ...
    
    # __repr__() replicates the (as yet unspecified) implementation of __str__()
    def __repr__(self):
        return str(self)
    
    # Even if id() and __mul__() are not defined yet, we can define power in terms of these
    def __pow__(a, n):
        if (n==0):
            return a.__class__.id()
        else:
            return a * (a**(n-1))

As you can see, we cannot create an instance of this class:

In [ ]:
a = Ring()

In fact, we have not even defined the methods such as `__mul__()`, `__add__()` etc. The current definition of `__mul__()`

```Python
    @abstractmethod
    def __mul__(a,b):
        ...
```

tells us the name of the method, `__mul__()`, and the fact that it should take two parameters `a` and `b`. However, it does not tell us how to compute the product `a * b`! Instead there are three dots `...` where the body (i.e. the implementation) of the method should be. 

Note that the line before the definition of `__mul__()` is `@abstractmethod`. A keyword appearing in the format `@keyword` in the line above a methods (or functions) definition is called a *decorator*. The decorator `@abstractmethod` tells Python 

* That `__mul__()` is just an abstract method with a specified syntax (`__mul__(a,b)`) and it does not have an implementation.
* That a class derived from `Ring` must implement `__mul__()`. 
* Only once all *abstract methods* have been implemented in a Derived Class will it be possible for an instance of the Derived Class to be created.

Note that every *object* has an attribute `__class__`. By accessing this class, we can access its methods and attributes. For instance, consider the following code:

```Python
    def __pow__(a, n):
        if (n==0):
            return a.__class__.id()
        else:
            return a * (a**(n-1))
```
Since $a^0 = 1$, when we encounter the $n=0$ case, we find the class of `a` and return the `id()` (i.e. the multiplicative identity $1$) belonging to this class. Note that the above definition of power will work as soon as we define `id` and `*` (i.e. `__mul__()`). Note that `a**(n-1)` is equivalent to `a.__pow__(n-1)`.


Note that we have violated the *self* notation in many places - sometimes using a simpler parameter name (such as `a` in `__pow__()`) is just much more natural. 

## Inheriting to another Abstract Class

We can inherit the abstract class `Ring` and create a concrete implementation. Or we can inherit to yet another abstract class. 

For instance, a *Field* is a *Ring* which also has 
* a multiplicative inverse for all elements (except for zero): for each $b \neq 0$, there is $b^{-1} \in A$ such that $b \cdot (b^{-1}) = 1$ 
* division $a/b = a \cdot (b^{-1})$.

Consider the previously introduced examples:

* $(\mathbb{Q}, +, \cdot, 0, 1)$ the ring of rationals is also a field.
* $(\mathbb{C}, +, \cdot, 0, 1)$ the ring of complex numbers is also a field.
* $(\mathbb{R}^{n\times n}, +, \cdot, O_n, I_n)$ the ring of $n \times n$ real-valued matrices is not a field since there are matrices (other than the zero matrix $O_n$) which do not have an inverse. 
* $(\mathcal{P}(x), +, \cdot, 0, 1)$ the ring of polynomials in $x$ is not a field since inverse of a polynomial is not a polynomial.

We define the class `Field` by inheriting from `Ring`:

In [ ]:
class Field(Ring):
    '''Field (A, +, ., zero, id)'''
    
    @abstractmethod
    def inv(a):
        ...
        
    # Even if inv() is not defined yet, and __mul__() is inherited but not defined, we can still define a/b = a * (b.inv())
    def __truediv__(a,b):
        return a*(b.inv())

Since we have not defined `inv()`, `id()`, `zero()`, `__mul__()` etc, `Field` is also an abstract class and we still cannot instantiate an object of type `Field`:

In [ ]:
a = Field()

This is not a limitation, but very much by design! The idea is that the `Field` class should act a *template* or a *blueprint* for specific fields such as $\mathbb{Q}$ and $\mathbb{C}$. 

Note that even though we have not specified how to compute `inv()` or `__mul__()` (i.e. `*`), we can define $a/b$ as $a \cdot b^{-1}$ or `__mul__(a, b.inv())` (equivalent to `a * b.inv())`). Later on, when we define `inv()` and `__mul__()` for each derived class, the appropriate functionality of `a/b` i.e. `__truediv__()` will automatically work.

# Implementing Abstract Classes and Abstract Methods

We now create a concrete example of a field: the field of rational numbers $(\mathbb{Q}, +, \cdot, 0, 1)$. We do so by deriving the new class `Rational` from the abstract class `Field` and implementing all the methods in `Field` and `Ring` which were decorated by `@abstractmethod`. 

In [ ]:
class Rational(Field):
    '''Field of rational numbers (Q, +, ., zero, id)'''
    
    def __init__(self,num,denom):
        self.num = num
        self.denom = denom
    
    def __str__(self):
        return '('+str(self.num)+'/'+str(self.denom)+')'
    
    def id():
        return Rational(1,1)
    
    def zero():
        return Rational(0,1)
    
    def __mul__(a,b):
        if isinstance(b,int):
            return Rational(a.num*b, a.denom)
        else:
            return Rational(a.num*b.num, a.denom*b.denom)
    
    def __add__(a,b):
        cnum = a.num*b.denom + b.num*a.denom
        cdenom = a.denom*b.denom
        return Rational(cnum, cdenom)
    
    def __neg__(a):
        return Rational(-a.num, a.denom)
    
    def inv(a):
        return Rational(a.denom, a.num)

`Rational` is a concrete class since all abstract methods have been implemented! We can create its instances:

In [ ]:
a = Rational(1,2)
b = Rational(3,5)
print(a)
print(b)
print(a+b)

We have not explicitly defined `__repr__()`,`__sub__()`, `__truediv__()` and `__pow__()` in `Rational`: These were already defined in `Ring` and `Field`, and are inherited. Note carefully, that these methos utilise the definitions of `__str__()`, `__add__()`, `__neg__()`, `__mul__()`, `inv()` and `id()`, which we defined in `Rational`.

Here is an example of `__repr__()` which enables us to display an object directly on the Notebook without `print`:

In [ ]:
a

Similarly, `__sub__()`, `__truediv__()` and `__pow__()` work as expected:

In [ ]:
print(a-b)
print(a/b)
print(a**4)

Due to operator overloading, we can work with instances of `Rational` in the same way as we are used to dealing with `float` and `int`, and can write some fairly complicated operations in a compact way:

In [ ]:
print(((a-b)**2)*(a+b)/b)

In [ ]:
import numpy as np
x1 = 1
y1 = 3
x2 = 0
y2 = -1
x3 = 7
y3 = 2
v = np.array([[x1, y1], [x2, y2], [x3, y3]])  
print(v)
print(v[0,0])
print(v[0,1])

## Check your understanding

The solutions to these excercises are provided at the very end of this notebook.

Consider the classes:

```Python
from abc import ABC, abstractmethod
import numpy as np

class Shape(ABC):
    
    @abstractmethod
    def perimeter(self):
        ...
    
    @abstractmethod
    def area(self):
        ...
        
        
class Circle(Shape):
    def __init__(self, centre_x, centre_y, radius):
        self.centre_x = centre_x
        self.centre_y = centre_y
        self.radius = radius
        
    def perimeter(self):
        return 2 * np.pi * self.radius
    
    def area(self):
        return np.pi * (self.radius**2)
        
                 
class Polygon(Shape):    
    def __init__(self, vertices):
        self.vertices = vertices

        
class Triangle(Polygon):
    def __init__(self, x1, y1, x2, y2, x3, y3):
        vertices = np.array([[x1, y1], [x2, y2], [x3, y3]])  
        super().__init__(vertices)
    
    def sides(self):
        v = self.vertices
        a = np.sqrt((v[0,0]-v[1,0])**2 + (v[0,1]-v[1,1])**2)
        b = np.sqrt((v[1,0]-v[2,0])**2 + (v[1,1]-v[2,1])**2)
        c = np.sqrt((v[2,0]-v[0,0])**2 + (v[2,1]-v[0,1])**2)
        return a, b, c
                
    def perimeter(self):
        # Compute lengths of sides
        [a, b, c] = self.sides()
        return a + b + c
    
    def area(self):
        # Compute lengths of sides
        [a, b, c] = self.sides()
        p = (a+b+c)/2
        return np.sqrt(p*(p-a)*(p-b)*(p-c))
          
```

**Q1)** Which of the following are derived classes? (multiple answers may be true)

a. `Shape` 

b. `Circle` 

c. `Polygon` 

d. `Triangle` 

<br>

**Q2)** Which of the following are abstract classes? (multiple answers may be true)

a. `Shape` 

b. `Circle` 

c. `Polygon` 

d. `Triangle` 

<br>

**Q3)**  Which of the following methods are abstract methods in the class `Polygon`? (multiple answers may be true)

a. `perimeter()` 

b. `area()` 

c. `__init__()` 

d. `sides()` 

<br>

**Q4)** We wish to create a class for equilateral triangles called `EquilateralTriangle`. Which class should `EquilateralTriangle` be derived from? Specify the most efficient option in the sense that the number of methods that have to be explicitly defined (effectively, overloaded) in `EquilateralTriangle` should be minimal. Also specify the (minimum) number of methods to be implemented. 

a. `Shape`, 4 methods

b. `Shape`, 2 methods

c. `Circle`, 3 methods

d. `Polygon`, 4 methods

e. `Polygon`, 3 methods

f. `Polygon`, 2 methods

g. `Triangle`, 4 methods

h. `Triangle`, 2 methods

i. `Triangle`, 1 method

<br>

**Q5)** We update the definition of the class `Polygon` by implementing the follwing method for computing perimeter-to-area ratio in the class `Polygon`:

```Python
    def perimeter_to_area(self):
        return self.perimeter()/self.area()
```

Which of the following classes have this method? (multiple answers may be true)

a. `Shape`

b. `Circle`

c. `Polygon`

d. `Triangle`

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Solutions to "Check your understanding"

**Q1)** Answer: b, c, d

**Q2)** Answer: a, c

**Q3)** Answer: a, b

**Q4)** Answer: i

**Q5)** Answer: c, d